# 데이터 불러오기 (학습 준비 단계)

### GPU 사용 확인

만약 LogicalDevice에 GPU가 표시되지 않는다면, CUDA와 cuDNN을 설치하여 gpu가 표시되도록 준비합니다.

텐서플로우 학습을 진행할때 gpu가 아닌 cpu를 사용한다면, 학습시간이 상당히 오래 걸릴 수 있습니다.

설치방법 https://uwgdqo.tistory.com/341

In [3]:
import tensorflow as tf 
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

## 이미지 불러오기 (baseline 코드)

In [4]:
import os
import warnings
warnings.filterwarnings(action='ignore')

os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU 할당

In [5]:
#create training dataset
from glob import glob
import numpy as np
import PIL
from PIL import Image

path = './data/train/'

training_images = []
training_labels = []

for filename in glob(path +"*"):
    for img in glob(filename + "/*.jpg"):
        an_img = PIL.Image.open(img) #read img
        img_array = np.array(an_img) #img to array
        training_images.append(img_array) #append array to training_images
        label = filename.split('\\')[-1] #get label
        training_labels.append(label) #append label
        
training_images = np.array(training_images)
training_labels = np.array(training_labels)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
training_labels= le.fit_transform(training_labels)
training_labels = training_labels.reshape(-1,1)

print(training_images.shape)
print(training_labels.shape)

(50000, 32, 32, 3)
(50000, 1)


파일 경로를 /data/train에서 불러옵니다. (조금 시간이 걸릴 수 있습니다)

여기서 label은 image가 저장된 폴더의 이름으로 설정되며, 총 50000개의 train image가 존재하는것을 알 수 있습니다.

In [6]:
#create test dataset

path = './data/test/'

test_images = []
test_idx = []

flist = sorted(glob(path + '*.jpg'))

for filename in flist:
    an_img = PIL.Image.open(filename) #read img
    img_array = np.array(an_img) #img to array
    test_images.append(img_array) #append array to training_images 
    label = filename.split('\\')[-1] #get id 
    test_idx.append(label) #append id
    
test_images = np.array(test_images)

print(test_images.shape)
print(test_idx[0:5])

(10000, 32, 32, 3)
['0000.jpg', '0001.jpg', '0002.jpg', '0003.jpg', '0004.jpg']


파일 경로를 /data/test에서 불러옵니다. (조금 시간이 걸릴 수 있습니다)

여기서 label은 image가 저장된 폴더의 이름으로 설정되며, 총 10000개의 train image가 존재하는 것을 알 수 있습니다.

# 데이터 전처리

## data augmentation을 통한 데이터 생성

vertical_flip은 사용하지 않습니다. Mixup은 TTA를 위해서 사용하지 않았습니다.

Test Time Augmention 및 Data Augmentation의 강도를 정할때 아래 사이트의 의견을 참고했습니다.

https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

tf.random.set_seed(42)

image_generator = ImageDataGenerator(
    rotation_range=20,
    brightness_range = [0.6, 1.0],
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

In [8]:
training_image_aug = image_generator.flow(training_images, np.zeros(50000), batch_size=50000, shuffle=False, seed = 42).next()[0]
training_image_aug_2 = image_generator.flow(training_images, np.zeros(50000), batch_size=50000, shuffle=False, seed = 42^2).next()[0]
training_image_aug_3 = image_generator.flow(training_images, np.zeros(50000), batch_size=50000, shuffle=False, seed = 42^3).next()[0]
training_image_aug_4 = image_generator.flow(training_images, np.zeros(50000), batch_size=50000, shuffle=False, seed = 42^4).next()[0]

**시드는 42 * i (i는 1~4까지)로 고정하여 나중에 TTA를 진행할때도 똑같이 적용해주도록 합니다.**

In [9]:
training_images = np.concatenate((training_images, 
                                  training_image_aug, 
                                  training_image_aug_2, 
                                  training_image_aug_3, 
                                  training_image_aug_4))

training_labels = np.concatenate((training_labels, 
                                  training_labels, 
                                  training_labels, 
                                  training_labels, 
                                  training_labels))

training_labels = tf.one_hot(training_labels, 10) #one-hot 기법 적용
training_labels = np.array(training_labels)
training_labels = training_labels.reshape(-1,10) #one-hot 기법을 적용했다면, shape을 바꿔줍니다.

In [10]:
print(training_images.shape)
print(training_labels.shape)

(250000, 32, 32, 3)
(250000, 10)


DataGenerator 함수를 이용해 5만개였던 train data가 총 25만개로 증가한 모습입니다.

### Data set 나누기

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(training_images, 
                                                      training_labels, 
                                                      test_size=0.05, 
                                                      stratify = training_labels, 
                                                      random_state=42,
                                                      shuffle = True)

X_test = test_images

In [12]:
print('X_train 크기:',X_train.shape)
print('y_train 크기:',y_train.shape)
print('X_valid 크기:',X_valid.shape)
print('y_valid 크기:',y_valid.shape)
print('X_test  크기:',X_test.shape)

X_train 크기: (237500, 32, 32, 3)
y_train 크기: (237500, 10)
X_valid 크기: (12500, 32, 32, 3)
y_valid 크기: (12500, 10)
X_test  크기: (10000, 32, 32, 3)


In [13]:
X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0

# 모델 생성하기

resnet50의 유형을 참고하고, bottleneck를 사용하지 않았습니다. (사실상 bottleneck 구조를 버려서 block에 3개의 conv가 있는 resnet 18(?)이라고도 할 수 있을것 같습니다.)

CNN을 구현했을때 성능이 잘나온 과정을 참고해서 수정했기때문에 원본 resnet보다 성능이 많이 나오지 않았을수도 있습니다. 

사진 및 구조는 아래 블로그를 참조했습니다.

https://ganghee-lee.tistory.com/41

### Resnet의 identity block

원래는 bottleneck 구조를 그대로 해보려다(64->64->256), gpu 성능이 딸려서 bottleneck 구조가 아난 3개의 conv를 사용하는(64->64->64) 블럭으로 구현해봤습니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fcrj5v9%2FbtqBOrWkyBD%2Fyxk3PchJlnl25RRXYJ1vg0%2Fimg.png" alt="identity block"/>

In [14]:
def identity_block(X, filters, kernel_size):
    X_shortcut = X
    
    X = tf.keras.layers.Conv2D(filters, kernel_size, padding='SAME')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    X = tf.keras.layers.Conv2D(filters, kernel_size, padding='SAME')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    X = tf.keras.layers.Conv2D(filters, kernel_size, padding='SAME')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    
    # Add
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X

### Resnet의 convolutional block

64 -> 128 등 conv filter수가 증가하게 될때 원래는 0 padding이나 1x1 conv filter를 이용해서 x의 차원을 증가시켜주는데. 저는 똑같은 3x3 conv filter를 x에 적용해 보았습니다 (conv filter수가 증가할때 이 작업을 하지 않는다면 shape(차원)이 맞지 않아 error가 뜰 수 있습니다.)

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbQtwY4%2FbtqBSPHVY9d%2FXLSNe8537wDXwnrXBAjJ70%2Fimg.png" alt="identity block"/>

In [15]:
def convolutional_block(X, filters, kernel_size):
    X_shortcut = X
    
    X = tf.keras.layers.Conv2D(filters, kernel_size, padding='SAME')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    X = tf.keras.layers.Conv2D(filters, kernel_size, padding='SAME')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    X = tf.keras.layers.Conv2D(filters, kernel_size, padding='SAME')(X)
    X = tf.keras.layers.BatchNormalization()(X)

    X_shortcut = tf.keras.layers.Conv2D(filters, kernel_size, padding='SAME')(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization()(X_shortcut)
    
    # Add
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X

### Resnet 50 Custom Light ver

4번의 stage와 stage마다 1개의 conv block와 1개의 identity block을 사용했습니다.

Total params: 18,805,962

In [16]:
def ResNet50CL(input_shape = (32, 32, 3), classes = 10):
    X_input = tf.keras.layers.Input(input_shape)
    X = X_input
    
    X = convolutional_block(X, 64, (3,3)) #conv
    X = identity_block(X, 64, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = convolutional_block(X, 128, (3,3)) #64->128, use conv block
    X = identity_block(X, 128, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = convolutional_block(X, 256, (3,3)) #128->256, use conv block
    X = identity_block(X, 256, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = convolutional_block(X, 512, (3,3)) #256->512, use conv block
    X = identity_block(X, 512, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = tf.keras.layers.GlobalAveragePooling2D()(X)
    X = tf.keras.layers.Dense(10, activation = 'softmax')(X) # ouput layer (10 class)

    model = tf.keras.models.Model(inputs = X_input, outputs = X, name = "ResNet50CL")
    
    return model

### Resnet 50 Custom (add one more block)

4번의 stage와 stage마다 1개의 conv block와 2개의 identity block을 사용했습니다.

Total params: 28,293,002

In [22]:
def ResNet50C(input_shape = (32, 32, 3), classes = 10):
    X_input = tf.keras.layers.Input(input_shape)
    X = X_input
    
    X = tf.keras.layers.Conv2D(64, (3,3), padding='SAME')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    X = convolutional_block(X, 64, (3,3)) #use conv block (?)
    X = identity_block(X, 64, (3,3))
    X = identity_block(X, 64, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = convolutional_block(X, 128, (3,3)) #64->128, use conv block
    X = identity_block(X, 128, (3,3))
    X = identity_block(X, 128, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = convolutional_block(X, 256, (3,3)) #128->256, use conv block
    X = identity_block(X, 256, (3,3))
    X = identity_block(X, 256, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = convolutional_block(X, 512, (3,3)) #256->512, use conv block
    X = identity_block(X, 512, (3,3))
    X = identity_block(X, 512, (3,3))
    X = tf.keras.layers.MaxPooling2D(2, 2, padding='SAME')(X)
    
    X = tf.keras.layers.GlobalAveragePooling2D()(X)
    X = tf.keras.layers.Dense(10, activation = 'softmax')(X) # ouput layer (10 class)

    model = tf.keras.models.Model(inputs = X_input, outputs = X, name = "ResNet50C")
    
    return model

# 모델 학습하기

## ResNet50CL (public 0.9092 when use TTA)

In [24]:
model = ResNet50CL()
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [25]:
EPOCH = 50
BATCH_SIZE = 128

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=10, 
                             )

reduceLR = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=4,
)

In [ ]:
data = model.fit(X_train, 
                 y_train, 
                 validation_data=(X_valid, y_valid), 
                 epochs=EPOCH, 
                 batch_size=BATCH_SIZE, 
                 callbacks=[reduceLR, earlystopping],)

In [ ]:
import matplotlib.pyplot as plot

plot.plot(data.history['accuracy'])
plot.plot(data.history['val_accuracy'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

plot.plot(data.history['loss'])
plot.plot(data.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

In [ ]:
model.save('ResNet50CL.h5')

# 모델 학습하기

## ResNet50C (public 0.8996 when use TTA)

In [ ]:
model = ResNet50C()
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
EPOCH = 50
BATCH_SIZE = 128

filename = 'resnet50C-checkpoint.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filename,             # file명을 지정합니다
                                                monitor='val_accuracy',   # val_accuracy 값이 개선되었을때 호출됩니다
                                                verbose=1,            # 로그를 출력합니다
                                                save_best_only=True,  # 가장 best 값만 저장합니다
                                                mode='auto'           # auto는 알아서 best를 찾습니다. min/max (loss->min, accuracy->max)
                                               )

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', #stop 조건으로 관찰할 변수 선택
                                                 patience=10,            #10 Epoch동안 (val-accuracy가)개선되지 않는다면 종료
                                                )

reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', #lr을 낮출 조건으로 관찰할 변수 선택
                                                factor=0.5,             #조건이 충족되었을때 LR에 factor를 곱함 (2분의 1배가 됨)
                                                patience=6,             #10 Epoch동안 (val-accuracy가)개선되지 않는다면 lr 감소
                                               )

ModelCheckpoint = 가장 'monitor'값이 좋았던, 즉 val_accuracy가 가장 높았던 모델을 filename으로 저장한다.

EarlyStopping = 'monitor'값, 즉 val_accuracy가 patience동안 개선되지 않았다면 학습을 종료한다.

ReduceLROnPlateau = 'monitor'값, 즉 val_accuracy가 patience동안 개선되지 않았다면 learning rate에 factor를 곱해준다 (0.5의 경우 반토막)

In [ ]:
data = model.fit(X_train, 
                 y_train, 
                 validation_data=(X_valid, y_valid), 
                 epochs=EPOCH, 
                 batch_size=BATCH_SIZE, 
                 callbacks=[reduceLR, earlystopping, checkpoint],)

In [ ]:
import matplotlib.pyplot as plot

plot.plot(data.history['accuracy'])
plot.plot(data.history['val_accuracy'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

plot.plot(data.history['loss'])
plot.plot(data.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

# 제출하기 (public 0.915 (ResNet50CL + ResNet50C))

학습된 모델을 이용하여 제출합니다.

TTA 기법을 사용하고, ResNet50CL과 ResNet50C를 앙상블한 결과를 제출합니다.

TTA = Test Time Augmentation, train data에 우리가 augmentation을 적용하여 학습한것처럼 test data도 augmentation을 적용해서 추론한 예측값으로 최종 예측을 내는 기법.

soymilk님의 사용법을 참고했습니다.

https://dacon.io/competitions/official/235874/codeshare/4575?page=1&dtype=recent

In [ ]:
model = tf.keras.models.load_model('./resnet50CL.h5') #학습했던 Resnet50CL 불러오기

In [20]:
X_test_ori = test_images
X_test = test_images
X_test = X_test / 255.0

pred_proba = model.predict(X_test)

#TTA 적용
for i in [1, 2, 3, 4]:
    X_test_aug = image_generator.flow(X_test_ori, np.zeros(10000), batch_size=10000, shuffle=False, seed = 42^i).next()[0]
    X_test_aug = X_test_aug / 255.0
    pred_proba_aug = model.predict(X_test_aug)
    pred_proba = np.add(pred_proba, pred_proba_aug)
    
pred_class = []

for i in pred_proba:
    pred = np.argmax(i)
    pred_class.append(pred)
    
pred_class = le.inverse_transform(pred_class)
pred_class[0:5]

array(['horse', 'bird', 'airplane', 'horse', 'airplane'], dtype='<U10')

In [ ]:
model = tf.keras.models.load_model('./resnet50C-checkpoint.h5') #학습했던 Resnet50C 불러오기

In [32]:
pred_proba_2 = model.predict(X_test)
pred_proba = np.add(pred_proba, pred_proba_2) #resnet50CL 결과에 추론결과를 계속 더함

#TTA 적용
for i in [1, 2, 3, 4]:
    X_test_aug = image_generator.flow(X_test_ori, np.zeros(10000), batch_size=10000, shuffle=False, seed = 42^i).next()[0]
    X_test_aug = X_test_aug / 255.0
    pred_proba_aug = model.predict(X_test_aug)
    pred_proba = np.add(pred_proba, pred_proba_aug)
    
pred_class = []

for i in pred_proba:
    pred = np.argmax(i)
    pred_class.append(pred)
    
pred_class = le.inverse_transform(pred_class)
pred_class[0:5]

array(['horse', 'bird', 'airplane', 'horse', 'airplane'], dtype='<U10')

In [33]:
import pandas as pd

sample_submission = pd.read_csv("./data/sample_submission.csv")

sample_submission.target = pred_class
sample_submission.to_csv("submit_25.csv",index=False)

In [34]:
sample_submission.head(10)

,id,target
0,0000.jpg,horse
1,0001.jpg,bird
2,0002.jpg,airplane
3,0003.jpg,horse
4,0004.jpg,airplane
5,0005.jpg,deer
6,0006.jpg,airplane
7,0007.jpg,deer
8,0008.jpg,airplane
9,0009.jpg,horse


# 여담

<img src="https://miro.medium.com/max/1400/1*chbylvv0Lts1hKEuOJix6g.png" alt="ResNet-CIFAR-10"/>

Resnet 논문을 보면 CIFAR - 10에 사용한 구조는 이런 식으로 다르다고 합니다. (사진의 해상도가 다르기 때문에)

https://towardsdatascience.com/resnets-for-cifar-10-e63e900524e0

32 x 32의 사이즈로 이미지가 작기 때문에 32 x 32 output부터 building block을 적용하는 모습입니다. 또한 3번의 stage를 거치고 바로 avg pool을 이용하여 결과를 추론하는 모습입니다. 

하지만 우리가 모델을 직접 구현하지 않고 불러오게 되면 기본적으로 ImageNet에서 사용하던 모델을 불러오기 때문에 32 x 32의 output을 활용하지 않고 pooling후 바로 16 x 16부터 계산하게 됩니다. (여기서 32 x 32, 16 x 16 output을 제대로 활용하지 못해서 얻는 손실이 있다고 생각했습니다.)

이러한 문제점 때문에 직접 구현해보게 되었습니다.

In [27]:
model_imp = tf.keras.applications.resnet50.ResNet50(include_top=True, weights=None, input_shape=(32,32,3))
model_imp.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

keras에서 불러온 ResNet50을 보면 conv 연산은 16 x 16 image부터 하는 모습입니다 (conv1_conv (Conv2D) (None, 16, 16, 64)) 

또한 16 x 16 에는 conv을 1번만 진행한 후, 8 x 8에서부터 본격적인 block을 사용하고 있습니다.